In [2]:
from pyxdf import match_streaminfos, resolve_streams
from mnelab.io.xdf import read_raw_xdf
import mne
import mnelab
from mne.time_frequency import csd_morlet, csd_fourier
from mne.time_frequency import tfr_morlet
import numpy as np
import matplotlib.pyplot as plt
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_test
import os.path
import pandas as pd
import matplotlib.patches as patches
import scipy
from EEGModels import EEGNet, ShallowConvNet, DeepConvNet
import pickle

In [6]:
reject_criteria = dict(eeg=100e-6)   

def read_file(raw):
    raw.filter(2, 40)
    raw2 =raw.copy().resample(128)
    ica = mne.preprocessing.ICA(n_components = 4, random_state = 0)
    ica.fit(raw2)
    raw2 = ica.apply(raw2,exclude = [0])
    events, _ = mne.events_from_annotations(raw2)
    # "Non distraction + congruent":6, "distraction + congruent" :4 , "Non distraction + incongruent":7, "distraction + incongruent" :5
    event_ids = {"Non distraction + congruent":6, "distraction + congruent" :4}
    epochs = mne.Epochs(raw2, events,event_id = event_ids,  preload=True,tmin = -3, tmax=1.0 , baseline=(None, 0),  reject=reject_criteria)
    labels=epochs.events[:,-1]
    features=epochs.get_data()
    labels[labels == 4] = 1 # distracted
    #labels[labels == 5] = 1 # distracted
    labels[labels == 6 ] = 0 # nd
    #labels[labels == 7] = 0 # nd

    # epoch_incongruent_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + congruent"],  preload=True,tmin = -0.5, tmax=0.8 , baseline=(None, 0))
    # epoch_incongruent_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + congruent"],  preload=True,tmin = -0.5, tmax=0.8, baseline=(None, 0))

    # epoch_incongruent_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    # epoch_incongruent_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5, baseline=(None, 0))

    # epoch_D = mne.Epochs(raw2, events,event_id = event_ids["distraction + congruent", "distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    # epoch_ND = mne.Epochs(raw2, events,event_id = event_ids["Non distraction + congruent", "distraction + incongruent"],  preload=True,tmin = -0.5, tmax=0.5 , baseline=(None, 0))
    return features,labels

Event ids are, 41 =  D + congruent, 42 =  D + incongruent,  51 =  ND + congruent, 52 =  ND + incongruent

In [4]:
mne.set_log_level(verbose='WARNING', return_old_level=False, add_frames=None)

In [7]:
streams = resolve_streams('sub-P001_ses-S001_task-Default_run-001_eeg.xdf')
stream_id = match_streaminfos(streams, [{"type": "signal"}])[0]
raw = read_raw_xdf('sub-P001_ses-S001_task-Default_run-001_eeg.xdf', stream_ids=[stream_id]).resample(250)

# features, labels, groups=[],[],[]
# for pnum in range(1,21):
#     paraticipant = "P"+ str(pnum)
#     print(pnum) 

#     path ='./'+ paraticipant +'/'
#     raw = None 
#     for num in range(1,5):
#         fileNamei = paraticipant +'_b'  + str(num) + ".xdf"
#         path_addr = path + fileNamei
#         if (os.path.exists(path_addr)):
#             streams = resolve_streams(path_addr)
#             stream_id = match_streaminfos(streams, [{"type": "EEG"}])[0]
#             rawi = read_raw_xdf(path_addr, stream_ids=[stream_id]).resample(250)
            
#             if (raw is not None):
#                 mne.io.concatenate_raws([raw, rawi], on_mismatch="ignore")
#             else:
#                 raw = rawi    

#     if (raw is not None):
#         feature_sess1,label_sess1= read_file(raw)
#         features.append(feature_sess1)
#         labels.append(label_sess1)
#         groups.append([pnum]*(len(label_sess1)))

Exception: file sub-P001_ses-S001_task-Default_run-001_eeg.xdf does not exist.

In [ ]:

# PATH =  path ='./exp7/'
# # ['TP9', 'AF7', 'AF8', 'TP10']
# chan = "TP10"
# nums = 6
# numOfPilots = 2

# features, labels, groups=[],[],[]
# for num in range(0,nums):
#     file_addr = path +"P"+ str(num) 
#     file_addr_session1 =   file_addr + "_sess1.xdf"
#     file_addr_session2 =   file_addr + "_sess2.xdf"
#     print(file_addr)

#     if (os.path.exists(file_addr_session1)):
#         print(file_addr_session1)
#         feature_sess1,label_sess1= read_file(file_addr_session1)
#         features.append(feature_sess1)
#         labels.append(label_sess1)
#         groups.append([num]*(len(label_sess1)))

#     if (os.path.exists(file_addr_session2)):
#         print(file_addr_session2)
#         feature_sess2,label_sess2= read_file(file_addr_session2)
#         features.append(feature_sess2)
#         labels.append(label_sess2)
#         groups.append([num]*(len(label_sess2)))
       
  
       

In [ ]:
# import numpy as np
# data_array=np.concatenate(features)
# labels=np.concatenate(labels)
# groups=np.concatenate(groups)

# features.shape,labels.shape,groups.shape

In [ ]:
data_array=np.vstack(features)
label_array=np.hstack(labels)
group_array=np.hstack(groups)
print(data_array.shape,label_array.shape,group_array.shape)

(1069, 4, 513) (1069,) (1069,)


In [ ]:
with open('congruent_features_bas_150.pickle', 'wb') as output:
    pickle.dump(data_array, output)


with open('congruent_labels_bas_150.pickle', 'wb') as output:
    pickle.dump(label_array, output)


with open('congruent_groups_bas_150.pickle', 'wb') as output:
    pickle.dump(group_array, output)    




In [ ]:
np.unique(group_array, axis=0)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18.])

LR

In [ ]:
# from scipy import stats
# def mean(data):
#     return np.mean(data,axis=-1)
    
# def std(data):
#     return np.std(data,axis=-1)

# def ptp(data):
#     return np.ptp(data,axis=-1)

# def var(data):
#         return np.var(data,axis=-1)

# def minim(data):
#       return np.min(data,axis=-1)


# def maxim(data):
#       return np.max(data,axis=-1)

# def argminim(data):
#       return np.argmin(data,axis=-1)


# def argmaxim(data):
#       return np.argmax(data,axis=-1)

# def mean_square(data):
#       return np.mean(data**2,axis=-1)

# def rms(data): #root mean square
#       return  np.sqrt(np.mean(data**2,axis=-1))  

# def abs_diffs_signal(data):
#     return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)


# def skewness(data):
#     return stats.skew(data,axis=-1)

# def kurtosis(data):
#     return stats.kurtosis(data,axis=-1)

# def concatenate_features(data):
#     return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
#                           mean_square(data),rms(data),abs_diffs_signal(data),
#                           skewness(data),kurtosis(data)),axis=-1)

In [ ]:
# from tqdm import tqdm_notebook
# features=[]
# for data in tqdm_notebook(data_array):
#     features.append(concatenate_features(data))
# features=np.array(features)
# features.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 

In [ ]:
# group_array.shape
# group_array

In [ ]:
# np.unique(labels) #7 left 8 right

In [ ]:
# unique, counts = np.unique(groups, return_counts=True)
# unique, counts

In [ ]:
# from mne.decoding import CSP

# # clf=svm.SVC()
# gkf=GroupKFold(n_splits=5)
# csp = CSP(n_components=3, norm_trace=False)
# param_grid = {'classifier__C': [0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]}
# pipe=Pipeline([('scaler',StandardScaler()), ("CSP", csp), ('classifier',clf)])
# gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=16)
# gscv.fit(features,label_array,groups=group_array)

In [ ]:
# gscv.best_score_
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()


CNN

In [ ]:
# data_array=np.vstack(data_array)
# label_array=np.hstack(label_array)
# group_array=np.hstack(group_array)
data_array_dl=np.moveaxis(data_array,1,2)

print(data_array.shape)

print(data_array_dl.shape,label_array.shape,group_array.shape)

(1069, 4, 513)
(1069, 513, 4) (1069,) (1069,)


In [ ]:
reshaped_data =  np.reshape(data_array, (4, data_array.shape[2],data_array.shape[0]))

print(data_array.shape)

(1069, 4, 513)


In [ ]:
#EEG NET
EEGNetModel  = EEGNet(nb_classes = 1, Chans = 4 , Samples = data_array.shape[2])
EEGNetModel.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])
EEGNetModel.summary()


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 4, 513, 1)]       0         
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 513, 8)         512       
                                                                 
 batch_normalization_12 (Bat  (None, 4, 513, 8)        32        
 chNormalization)                                                
                                                                 
 depthwise_conv2d_2 (Depthwi  (None, 1, 513, 16)       64        
 seConv2D)                                                       
                                                                 
 batch_normalization_13 (Bat  (None, 1, 513, 16)       64        
 chNormalization)                                                
                                                           

In [ ]:

accuracy=[]
for train_index, val_index in logo.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=EEGNetModel
    model.fit(train_features,train_labels,epochs=20,batch_size=96,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/20
11/11 [==============================] - 1s 41ms/step - loss: 0.6993 - accuracy: 0.5015 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 2/20
11/11 [==============================] - 0s 28ms/step - loss: 0.6886 - accuracy: 0.5015 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 3/20
11/11 [==============================] - 0s 28ms/step - loss: 0.6888 - accuracy: 0.5015 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 4/20
11/11 [==============================] - 0s 28ms/step - loss: 0.6856 - accuracy: 0.5015 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 5/20
11/11 [==============================] - 0s 28ms/step - loss: 0.6831 - accuracy: 0.5015 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 6/20
11/11 [==============================] - 0s 28ms/step - loss: 0.6855 - accuracy: 0.5015 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 7/20
11/11 [==============================] - 0s 30ms/step - loss: 0.6819 - accuracy: 0.5015 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 8/20
11

In [ ]:
np.mean(accuracy)

0.4998527980513043

In [ ]:
#shallownet
shallowNetModel  = ShallowConvNet(nb_classes = 1, Chans = 4 , Samples = data_array.shape[2])
shallowNetModel.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])
shallowNetModel.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 4, 513, 1)]       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 4, 501, 40)        560       
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 501, 40)        6400      
                                                                 
 batch_normalization_15 (Bat  (None, 1, 501, 40)       160       
 chNormalization)                                                
                                                                 
 activation_17 (Activation)  (None, 1, 501, 40)        0         
                                                                 
 average_pooling2d_8 (Averag  (None, 1, 67, 40)        0         
 ePooling2D)                                               

In [ ]:

accuracy=[]
for train_index, val_index in logo.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=shallowNetModel
    model.fit(train_features,train_labels,epochs=50,batch_size=64,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/50
16/16 [==============================] - 1s 68ms/step - loss: 0.7564 - accuracy: 0.5015 - val_loss: 0.7365 - val_accuracy: 0.5000
Epoch 2/50
16/16 [==============================] - 0s 26ms/step - loss: 0.6819 - accuracy: 0.5015 - val_loss: 0.7225 - val_accuracy: 0.5000
Epoch 3/50
16/16 [==============================] - 0s 27ms/step - loss: 0.6528 - accuracy: 0.5015 - val_loss: 0.7138 - val_accuracy: 0.5000
Epoch 4/50
16/16 [==============================] - 0s 26ms/step - loss: 0.6294 - accuracy: 0.5015 - val_loss: 0.7151 - val_accuracy: 0.5000
Epoch 5/50
16/16 [==============================] - 0s 26ms/step - loss: 0.6174 - accuracy: 0.5015 - val_loss: 0.7936 - val_accuracy: 0.5000
Epoch 6/50
16/16 [==============================] - 0s 26ms/step - loss: 0.6053 - accuracy: 0.5015 - val_loss: 0.7834 - val_accuracy: 0.5000
Epoch 7/50
16/16 [==============================] - 0s 26ms/step - loss: 0.5731 - accuracy: 0.5015 - val_loss: 0.7294 - val_accuracy: 0.5000
Epoch 8/50
16

In [ ]:
np.mean(accuracy)

0.4998527980513043

In [ ]:
#shallownet
DeepNetModel  = DeepConvNet(nb_classes = 1, Chans = 4 , Samples = data_array.shape[2])
DeepNetModel.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])
DeepNetModel.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 4, 513, 1)]       0         
                                                                 
 conv2d_14 (Conv2D)          (None, 4, 509, 25)        150       
                                                                 
 conv2d_15 (Conv2D)          (None, 1, 509, 25)        2525      
                                                                 
 batch_normalization_16 (Bat  (None, 1, 509, 25)       100       
 chNormalization)                                                
                                                                 
 activation_20 (Activation)  (None, 1, 509, 25)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 1, 254, 25)       0         
 2D)                                                       

In [ ]:
accuracy=[]
for train_index, val_index in logo.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=DeepNetModel
    model.fit(train_features,train_labels,epochs=50,batch_size=64,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/50
16/16 [==============================] - 3s 92ms/step - loss: 0.7786 - accuracy: 0.5015 - val_loss: 0.6987 - val_accuracy: 0.5000
Epoch 2/50
16/16 [==============================] - 1s 77ms/step - loss: 0.7439 - accuracy: 0.5015 - val_loss: 0.6864 - val_accuracy: 0.5000
Epoch 3/50
16/16 [==============================] - 1s 77ms/step - loss: 0.7218 - accuracy: 0.5015 - val_loss: 0.7019 - val_accuracy: 0.5000
Epoch 4/50
16/16 [==============================] - 1s 80ms/step - loss: 0.6991 - accuracy: 0.5015 - val_loss: 0.7091 - val_accuracy: 0.5000
Epoch 5/50
16/16 [==============================] - 1s 79ms/step - loss: 0.6961 - accuracy: 0.5015 - val_loss: 0.6974 - val_accuracy: 0.5000
Epoch 6/50
16/16 [==============================] - 1s 82ms/step - loss: 0.7132 - accuracy: 0.5015 - val_loss: 0.7038 - val_accuracy: 0.5000
Epoch 7/50
16/16 [==============================] - 1s 81ms/step - loss: 0.7406 - accuracy: 0.5015 - val_loss: 0.7083 - val_accuracy: 0.5000
Epoch 8/50
16

In [ ]:
np.mean(accuracy)

0.4998527980513043

In [ ]:
# auto encoders 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
# This is the dimension of the original space (1553, 4, 167)
timesteps = 167
n_features = 4
input_dim = 167


encoder = Sequential([
    LSTM(256, activation='selu', input_shape=(timesteps,n_features), return_sequences=False),
#    LSTM(1024, activation='selu', return_sequences=False),
#    Dense(128, activation='relu'),
    RepeatVector(timesteps)
])

decoder = Sequential([
    LSTM(256, activation='selu', input_shape=(1,1024),return_sequences=True),
#    LSTM(1024, activation='relu', return_sequences=True),
    TimeDistributed(Dense(n_features))
])

In [ ]:
autoencoder = Model(inputs=encoder.input, outputs=decoder(encoder.output))
autoencoder.compile(loss='mean_absolute_error', optimizer='adam')

ValueError: Exception encountered when calling layer "sequential_1" (type Sequential).

Input 0 of layer "lstm_1" is incompatible with the layer: expected shape=(None, None, 1024), found shape=(None, 167, 256)

Call arguments received by layer "sequential_1" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 167, 256), dtype=float32)
  • training=False
  • mask=None

In [ ]:
np.mean(accuracy)

In [ ]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(326,4)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 324, 5)            65        
                                                                 
 batch_normalization (BatchN  (None, 324, 5)           20        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 324, 5)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 162, 5)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 160, 5)            80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 160, 5)            0

In [ ]:

accuracy=[]
for train_index, val_index in logo.split(data_array_dl, label_array, groups=group_array):
    train_features,train_labels=data_array_dl[train_index],label_array[train_index]
    val_features,val_labels=data_array_dl[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=64,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/50
23/23 [==============================] - 2s 31ms/step - loss: 0.6933 - accuracy: 0.4880 - val_loss: 0.6926 - val_accuracy: 0.5208
Epoch 2/50
23/23 [==============================] - 0s 11ms/step - loss: 0.6925 - accuracy: 0.5045 - val_loss: 0.6927 - val_accuracy: 0.5208
Epoch 3/50
23/23 [==============================] - 0s 10ms/step - loss: 0.6936 - accuracy: 0.4962 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 4/50
23/23 [==============================] - 0s 11ms/step - loss: 0.6944 - accuracy: 0.4887 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 5/50
23/23 [==============================] - 0s 10ms/step - loss: 0.6926 - accuracy: 0.5154 - val_loss: 0.6937 - val_accuracy: 0.3854
Epoch 6/50
23/23 [==============================] - 0s 10ms/step - loss: 0.6933 - accuracy: 0.4990 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 7/50
23/23 [==============================] - 0s 11ms/step - loss: 0.6925 - accuracy: 0.5065 - val_loss: 0.6941 - val_accuracy: 0.4896
Epoch 8/50
23

In [ ]:
np.mean(accuracy)

0.4918975055217743

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D # new!

In [ ]:
def cnnmodel_lenet():
    clear_session()
    model = Sequential()

    model.add(Conv1D(32, kernel_size=(3), activation='relu', input_shape=(326,4)))

    model.add(Conv1D(64, kernel_size=(3), activation='relu'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel_lenet()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 324, 32)           416       
                                                                 
 conv1d_1 (Conv1D)           (None, 322, 64)           6208      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 161, 64)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 161, 64)           0         
                                                                 
 flatten (Flatten)           (None, 10304)             0         
                                                                 
 dense (Dense)               (None, 128)               1319040   
                                                        

In [ ]:
accuracy=[]
for train_index, val_index in logo.split(data_array_dl, label_array, groups=group_array):
    train_features,train_labels=data_array_dl[train_index],label_array[train_index]
    val_features,val_labels=data_array_dl[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel_lenet()
    model.fit(train_features,train_labels,epochs=50,batch_size=64,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/50
23/23 [==============================] - 1s 22ms/step - loss: 0.7295 - accuracy: 0.5017 - val_loss: 0.6969 - val_accuracy: 0.5104
Epoch 2/50
23/23 [==============================] - 0s 11ms/step - loss: 0.6798 - accuracy: 0.5621 - val_loss: 0.7014 - val_accuracy: 0.4375
Epoch 3/50
23/23 [==============================] - 0s 10ms/step - loss: 0.6666 - accuracy: 0.5944 - val_loss: 0.7077 - val_accuracy: 0.4479
Epoch 4/50
23/23 [==============================] - 0s 12ms/step - loss: 0.6443 - accuracy: 0.6198 - val_loss: 0.7178 - val_accuracy: 0.4583
Epoch 5/50
23/23 [==============================] - 0s 14ms/step - loss: 0.6290 - accuracy: 0.6582 - val_loss: 0.7209 - val_accuracy: 0.3958
Epoch 6/50
23/23 [==============================] - 0s 12ms/step - loss: 0.5835 - accuracy: 0.7014 - val_loss: 0.7346 - val_accuracy: 0.4583
Epoch 7/50
23/23 [==============================] - 0s 12ms/step - loss: 0.5524 - accuracy: 0.7035 - val_loss: 0.7705 - val_accuracy: 0.4167
Epoch 8/50
23

In [ ]:
np.mean(accuracy)

0.48959304988384245